# Step 1: Exact Jaccard Similarity

## 1.1 Read the original matrix

In [1]:
from scipy.io import mmread
from sklearn.metrics import jaccard_score

Read the sparse matrix

In [2]:
mtx = mmread('bbcsport/bbcsport.mtx')

Convert the frequency matrix to binary

In [3]:
binary_mtx = mtx.copy()
binary_mtx.data = (binary_mtx.data > 0).astype(int)

Convert to CSR (compressed sparse rows) for slicing

In [4]:
binary_mtx = binary_mtx.tocsr()
binary_mtx.shape

(4613, 737)

## 1.2 Compute Exact Jaccard Similarity

In [5]:
similarity_mtx = []
for i in range(binary_mtx.shape[1]):
    for j in range(i+1, binary_mtx.shape[1]):
        sim = jaccard_score(binary_mtx[:, i].toarray().ravel(), binary_mtx[:, j].toarray().ravel(), 
                                             pos_label=1, average='binary')
        similarity_mtx.append(((i, j), sim))

# similarity_mtx

In [6]:
len(similarity_mtx)

271216

## 1.3 Exact similar pairs (threshold=0.5)

In [8]:
threshold = 0.5

similar_pairs = [(i,j) for ((i,j), sim) in similarity_mtx if sim >= threshold]

for (i,j) in similar_pairs:
    print((i,j))
print("Total number of pairs:")
print(len(similar_pairs))


(11, 19)
(48, 87)
(51, 83)
(104, 126)
(117, 121)
(123, 129)
(124, 132)
(125, 133)
(141, 214)
(142, 143)
(152, 153)
(160, 161)
(187, 188)
(226, 486)
(227, 434)
(242, 251)
(311, 312)
(356, 362)
(382, 385)
(387, 388)
(413, 429)
(495, 622)
(508, 520)
(518, 632)
(534, 535)
(538, 550)
(541, 624)
(591, 593)
(601, 602)
(633, 634)
(640, 727)
(641, 649)
(643, 725)
(646, 652)
(648, 654)
(650, 731)
(660, 674)
(683, 687)
(685, 686)
(694, 695)
(705, 706)
Total number of pairs:
41


Similar Pairs: (11, 19)
(48, 87)
(51, 83)
(104, 126)
(117, 121)
(123, 129)
(124, 132)
(125, 133)
(141, 214)
(142, 143)
(152, 153)
(160, 161)
(187, 188)
(226, 486)
(227, 434)
(242, 251)
(311, 312)
(356, 362)
(382, 385)
(387, 388)
(413, 429)
(495, 622)
(508, 520)
(518, 632)
(534, 535)
(538, 550)
(541, 624)
(591, 593)
(601, 602)
(633, 634)
(640, 727)
(641, 649)
(643, 725)
(646, 652)
(648, 654)
(650, 731)
(660, 674)
(683, 687)
(685, 686)
(694, 695)
(705, 706)

Total number of pairs:
41

# Step 2: Min-Hashing Implementation

**Hash function:**

$$h_i(r)=((a_i*r+b_i)\% c)\% N $$

## 2.1 Generate 50 hash functions.

N = 4613 = binary_mtx.shape[0]

c = 4759

a, b random integers smaller than N

In [7]:
import random
import numpy as np
np.random.seed(1013)

n_hash=50

a = []
b = []
c = 4759
# c = 4637
n = binary_mtx.shape[0]

def gen_a(max, n):
    a = []
    for i in range(n):
        a.append(random.randint(1, max))
    return a

a = gen_a(n-1, n_hash)
b = gen_a(n-1, n_hash)

print("a={}".format(a))
print("b={}".format(b))
print("c={}".format(c))

a=[2661, 2138, 3858, 3297, 2591, 538, 2795, 3625, 1340, 2352, 2631, 3578, 2678, 730, 1373, 499, 1134, 2461, 4580, 3254, 208, 3220, 2556, 2162, 2188, 1613, 923, 359, 2734, 2986, 4490, 2897, 3261, 2473, 2983, 3981, 4290, 1015, 1377, 3187, 309, 4281, 1661, 2779, 804, 1395, 3206, 2478, 1332, 450]
b=[3019, 4531, 2719, 2889, 2870, 1902, 3498, 3563, 2178, 3253, 3835, 2, 4222, 1328, 3431, 2235, 3919, 3177, 3983, 4267, 3347, 4408, 1039, 4309, 1941, 1296, 4032, 2080, 400, 1650, 2606, 3018, 1721, 391, 1548, 3645, 1492, 1607, 3987, 4412, 3735, 3670, 1216, 47, 188, 2216, 1173, 1980, 672, 2146]
c=4759


a=[2661, 2138, 3858, 3297, 2591, 538, 2795, 3625, 1340, 2352, 2631, 3578, 2678, 730, 1373, 499, 1134, 2461, 4580, 3254, 208, 3220, 2556, 2162, 2188, 1613, 923, 359, 2734, 2986, 4490, 2897, 3261, 2473, 2983, 3981, 4290, 1015, 1377, 3187, 309, 4281, 1661, 2779, 804, 1395, 3206, 2478, 1332, 450]

b=[3019, 4531, 2719, 2889, 2870, 1902, 3498, 3563, 2178, 3253, 3835, 2, 4222, 1328, 3431, 2235, 3919, 3177, 3983, 4267, 3347, 4408, 1039, 4309, 1941, 1296, 4032, 2080, 400, 1650, 2606, 3018, 1721, 391, 1548, 3645, 1492, 1607, 3987, 4412, 3735, 3670, 1216, 47, 188, 2216, 1173, 1980, 672, 2146]

c=4759

## 2.2 Compute Signature matrix

Initialize signiture matrix sig(i,c)

In [9]:
import math

def init_sig(n_hash, n_cols):
    sig = np.ones((n_hash, n_cols))*float('inf')
    return sig

sig = init_sig(n_hash, binary_mtx.shape[1])
sig.shape


(50, 737)

Compute sig matrix

In [10]:
h = np.zeros(n_hash)
for r in range(n):
    for i in range(n_hash):
        h[i] = ((a[i]*r+b[i])%c)%n
    for col in range(binary_mtx.shape[1]):
        if binary_mtx[r,col] == 1:
            for i in range(n_hash):
                if h[i] <= sig[i,col]:
                    sig[i,col] = h[i]

sig


array([[ 2.,  2.,  7., ..., 40., 29.,  2.],
       [42., 46., 28., ..., 19., 28., 13.],
       [16., 83., 11., ..., 18., 21., 19.],
       ...,
       [11., 11.,  0., ...,  3., 31., 21.],
       [16.,  3., 12., ..., 28., 12., 25.],
       [19.,  6., 16., ..., 19.,  6., 19.]])

In [11]:
sig.shape

(50, 737)

## 2.3 Calculate approximate Jaccard similarity

In [12]:
# The approximate sim score is the fraction of the hash functions in which they agree:
def approximate_sim_score(sig1, sig2):
    intersect = (sig1==sig2).astype(int)
    score = (intersect.sum())/len(sig1)
    return score


signature_sim = []

for i in range(binary_mtx.shape[1]):
    for j in range(i+1, binary_mtx.shape[1]):
        sim = approximate_sim_score(sig[:,i], sig[:,j])
        signature_sim.append(((i,j),sim))

signature_sim


[((0, 1), 0.22),
 ((0, 2), 0.06),
 ((0, 3), 0.04),
 ((0, 4), 0.12),
 ((0, 5), 0.16),
 ((0, 6), 0.18),
 ((0, 7), 0.08),
 ((0, 8), 0.16),
 ((0, 9), 0.14),
 ((0, 10), 0.12),
 ((0, 11), 0.12),
 ((0, 12), 0.12),
 ((0, 13), 0.14),
 ((0, 14), 0.14),
 ((0, 15), 0.04),
 ((0, 16), 0.04),
 ((0, 17), 0.14),
 ((0, 18), 0.12),
 ((0, 19), 0.12),
 ((0, 20), 0.08),
 ((0, 21), 0.08),
 ((0, 22), 0.06),
 ((0, 23), 0.12),
 ((0, 24), 0.12),
 ((0, 25), 0.02),
 ((0, 26), 0.1),
 ((0, 27), 0.12),
 ((0, 28), 0.14),
 ((0, 29), 0.14),
 ((0, 30), 0.24),
 ((0, 31), 0.1),
 ((0, 32), 0.02),
 ((0, 33), 0.08),
 ((0, 34), 0.04),
 ((0, 35), 0.1),
 ((0, 36), 0.12),
 ((0, 37), 0.12),
 ((0, 38), 0.08),
 ((0, 39), 0.02),
 ((0, 40), 0.06),
 ((0, 41), 0.08),
 ((0, 42), 0.08),
 ((0, 43), 0.1),
 ((0, 44), 0.08),
 ((0, 45), 0.04),
 ((0, 46), 0.1),
 ((0, 47), 0.08),
 ((0, 48), 0.04),
 ((0, 49), 0.06),
 ((0, 50), 0.06),
 ((0, 51), 0.12),
 ((0, 52), 0.1),
 ((0, 53), 0.04),
 ((0, 54), 0.18),
 ((0, 55), 0.06),
 ((0, 56), 0.14),
 ((0, 5

## 2.4 List similar pairs

In [13]:
similar_pairs_approximate = [(i,j) for ((i,j),sim) in signature_sim if sim >= threshold]

for (i,j) in similar_pairs_approximate:
    print((i,j))
print("Total number of pairs:")
print(len(similar_pairs_approximate))

(11, 19)
(32, 53)
(48, 87)
(51, 83)
(78, 80)
(104, 126)
(117, 121)
(123, 129)
(124, 132)
(125, 133)
(141, 214)
(142, 143)
(152, 153)
(160, 161)
(187, 188)
(226, 486)
(227, 434)
(242, 251)
(311, 312)
(382, 385)
(387, 388)
(413, 429)
(495, 622)
(506, 521)
(508, 520)
(518, 632)
(534, 535)
(538, 550)
(541, 624)
(591, 593)
(601, 602)
(633, 634)
(640, 727)
(641, 649)
(643, 725)
(646, 652)
(648, 654)
(650, 731)
(660, 674)
(683, 687)
(685, 686)
(694, 695)
(705, 706)
Total number of pairs:
43


Similar Pairs: 
(11, 19)
(32, 53)
(48, 87)
(51, 83)
(78, 80)
(104, 126)
(117, 121)
(123, 129)
(124, 132)
(125, 133)
(141, 214)
(142, 143)
(152, 153)
(160, 161)
(187, 188)
(226, 486)
(227, 434)
(242, 251)
(311, 312)
(382, 385)
(387, 388)
(413, 429)
(495, 622)
(506, 521)
(508, 520)
(518, 632)
(534, 535)
(538, 550)
(541, 624)
(591, 593)
(601, 602)
(633, 634)
(640, 727)
(641, 649)
(643, 725)
(646, 652)
(648, 654)
(650, 731)
(660, 674)
(683, 687)
(685, 686)
(694, 695)
(705, 706)

Total number of pairs:
43

# Step 3: Comparison

In [14]:
import sklearn
import sklearn.metrics
# True Positives:
TP = 0

def true_positive(target, pred):
    score = 0
    for value in pred:
        if value in target:
            score = score + 1
    return score

TP = true_positive(similar_pairs, similar_pairs_approximate)
print("True Positives: {}".format(TP))

# False Positives:
FP = 0

def false_positive(target, pred):
    score = 0
    for value in pred:
        if value not in target:
            score = score + 1
    return score

FP = false_positive(similar_pairs, similar_pairs_approximate)
print("False Positives: {}".format(FP))

# False Negatives:
FN = 0

def false_negative(target, pred):
    score = 0
    for value in target:
        if value not in pred:
            score = score + 1
    return score

FN = false_negative(similar_pairs, similar_pairs_approximate)
print("False Negatives: {}".format(FN))

# True Negatives:
TN = 0
TN = len(similarity_mtx) - TP - FN - FP
print("True Negatives: {}".format(TN))

# F1 score of minhashing approximation
f1_score = (2*TP)/(2*TP+FN+FP)
print("F1 score of approximation: {}".format(f1_score))


True Positives: 40
False Positives: 3
False Negatives: 1
True Negatives: 271172
F1 score of approximation: 0.9523809523809523


True Positives: 40

False Positives: 3

False Negatives: 1

True Negatives: 271172

F1 score of approximation: 0.9523809523809523

# Step 4: 100 hash functions

## 4.1 Compute similar pairs with 100 hash functions

In [61]:
# generate 100 hash functions
n_hash = 100
a = []
b = []
c = 4759

a = gen_a(n-1, n_hash)
b = gen_a(n-1, n_hash)

print("a={}".format(a))
print("b={}".format(b))
print("c={}".format(c))

a=[3901, 1015, 1512, 1035, 1372, 4444, 246, 1385, 3987, 627, 1139, 1671, 3495, 1207, 1835, 2994, 2753, 1268, 152, 3774, 3739, 3841, 1029, 1716, 194, 4543, 1897, 3833, 392, 332, 1181, 426, 105, 2331, 903, 4492, 3009, 4185, 3376, 4347, 2966, 4181, 599, 80, 1851, 892, 4364, 3254, 4351, 1940, 86, 2883, 2626, 4236, 3059, 3145, 1007, 1133, 2639, 3860, 4380, 97, 85, 2319, 972, 1076, 2823, 261, 2788, 44, 3453, 2203, 3572, 3511, 922, 2754, 556, 4143, 1027, 3873, 3792, 968, 3133, 3988, 4193, 315, 1989, 4015, 3282, 1435, 1355, 3996, 3911, 3007, 3058, 2820, 2673, 3180, 1036, 3148]
b=[2375, 2221, 76, 3791, 3306, 512, 1152, 2502, 2473, 3683, 1660, 15, 2118, 1919, 3161, 4264, 1943, 4303, 1984, 1745, 4428, 1118, 568, 2304, 2721, 1257, 2749, 2420, 3217, 2605, 2516, 2184, 2602, 604, 3439, 180, 788, 1144, 63, 3054, 3240, 1522, 4197, 1213, 1594, 1107, 1740, 4305, 4061, 2175, 2046, 3413, 1124, 3225, 1400, 4572, 2049, 3291, 3338, 1232, 4391, 156, 3898, 1531, 2087, 3938, 740, 1256, 180, 2474, 1556, 2830, 338

a=[3901, 1015, 1512, 1035, 1372, 4444, 246, 1385, 3987, 627, 1139, 1671, 3495, 1207, 1835, 2994, 2753, 1268, 152, 3774, 3739, 3841, 1029, 1716, 194, 4543, 1897, 3833, 392, 332, 1181, 426, 105, 2331, 903, 4492, 3009, 4185, 3376, 4347, 2966, 4181, 599, 80, 1851, 892, 4364, 3254, 4351, 1940, 86, 2883, 2626, 4236, 3059, 3145, 1007, 1133, 2639, 3860, 4380, 97, 85, 2319, 972, 1076, 2823, 261, 2788, 44, 3453, 2203, 3572, 3511, 922, 2754, 556, 4143, 1027, 3873, 3792, 968, 3133, 3988, 4193, 315, 1989, 4015, 3282, 1435, 1355, 3996, 3911, 3007, 3058, 2820, 2673, 3180, 1036, 3148]

b=[2375, 2221, 76, 3791, 3306, 512, 1152, 2502, 2473, 3683, 1660, 15, 2118, 1919, 3161, 4264, 1943, 4303, 1984, 1745, 4428, 1118, 568, 2304, 2721, 1257, 2749, 2420, 3217, 2605, 2516, 2184, 2602, 604, 3439, 180, 788, 1144, 63, 3054, 3240, 1522, 4197, 1213, 1594, 1107, 1740, 4305, 4061, 2175, 2046, 3413, 1124, 3225, 1400, 4572, 2049, 3291, 3338, 1232, 4391, 156, 3898, 1531, 2087, 3938, 740, 1256, 180, 2474, 1556, 2830, 338, 1143, 3186, 4169, 942, 3706, 1861, 1498, 42, 2792, 3075, 3149, 4410, 1220, 985, 274, 933, 3974, 4353, 3909, 2820, 1851, 37, 2724, 721, 1095, 2998, 2325]

c=4759

In [62]:
# initialize signiture matrix
sig = init_sig(n_hash, binary_mtx.shape[1])
sig.shape


(100, 737)

In [63]:
# compute signiture matrix
h = np.zeros(n_hash)
for r in range(n):
    for i in range(n_hash):
        h[i] = ((a[i]*r+b[i])%c)%n
    for col in range(binary_mtx.shape[1]):
        if binary_mtx[r,col] == 1:
            for i in range(n_hash):
                if h[i] <= sig[i,col]:
                    sig[i,col] = h[i]

sig

array([[ 27.,  52.,  49., ...,  57.,  15.,  38.],
       [  2.,  38.,  19., ...,  61.,  38.,  22.],
       [ 27.,  26.,   4., ...,  27.,  53.,  27.],
       ...,
       [  0.,   0.,  14., ...,   1.,  25.,  63.],
       [ 69.,  22.,   7., ...,  11.,  42.,  11.],
       [ 46.,  29.,  12., ..., 162.,  29.,   4.]])

In [64]:
# calculate signiture sim score:
signature_sim = []

for i in range(binary_mtx.shape[1]):
    for j in range(i+1, binary_mtx.shape[1]):
        sim = approximate_sim_score(sig[:,i], sig[:,j])
        signature_sim.append(((i,j),sim))

signature_sim

[((0, 1), 0.15),
 ((0, 2), 0.17),
 ((0, 3), 0.08),
 ((0, 4), 0.18),
 ((0, 5), 0.19),
 ((0, 6), 0.05),
 ((0, 7), 0.04),
 ((0, 8), 0.21),
 ((0, 9), 0.15),
 ((0, 10), 0.04),
 ((0, 11), 0.14),
 ((0, 12), 0.08),
 ((0, 13), 0.12),
 ((0, 14), 0.08),
 ((0, 15), 0.04),
 ((0, 16), 0.02),
 ((0, 17), 0.12),
 ((0, 18), 0.1),
 ((0, 19), 0.14),
 ((0, 20), 0.07),
 ((0, 21), 0.18),
 ((0, 22), 0.02),
 ((0, 23), 0.08),
 ((0, 24), 0.1),
 ((0, 25), 0.06),
 ((0, 26), 0.09),
 ((0, 27), 0.08),
 ((0, 28), 0.12),
 ((0, 29), 0.07),
 ((0, 30), 0.1),
 ((0, 31), 0.22),
 ((0, 32), 0.01),
 ((0, 33), 0.1),
 ((0, 34), 0.08),
 ((0, 35), 0.07),
 ((0, 36), 0.1),
 ((0, 37), 0.08),
 ((0, 38), 0.05),
 ((0, 39), 0.06),
 ((0, 40), 0.07),
 ((0, 41), 0.08),
 ((0, 42), 0.09),
 ((0, 43), 0.07),
 ((0, 44), 0.05),
 ((0, 45), 0.04),
 ((0, 46), 0.04),
 ((0, 47), 0.03),
 ((0, 48), 0.04),
 ((0, 49), 0.06),
 ((0, 50), 0.04),
 ((0, 51), 0.12),
 ((0, 52), 0.06),
 ((0, 53), 0.05),
 ((0, 54), 0.21),
 ((0, 55), 0.22),
 ((0, 56), 0.11),
 ((0, 

In [65]:
# list similar pairs:
similar_pairs_approximate = [(i,j) for ((i,j),sim) in signature_sim if sim >= threshold]

for (i,j) in similar_pairs_approximate:
    print((i,j))
print("Total number of pairs:")
print(len(similar_pairs_approximate))

(11, 19)
(22, 47)
(48, 87)
(51, 83)
(104, 126)
(117, 121)
(123, 129)
(124, 132)
(125, 133)
(141, 214)
(142, 143)
(152, 153)
(160, 161)
(187, 188)
(226, 486)
(227, 434)
(242, 251)
(311, 312)
(356, 362)
(382, 385)
(387, 388)
(413, 429)
(495, 622)
(506, 521)
(508, 520)
(518, 632)
(534, 535)
(538, 550)
(541, 624)
(591, 593)
(601, 602)
(633, 634)
(640, 727)
(641, 649)
(643, 725)
(646, 652)
(648, 654)
(650, 731)
(660, 674)
(683, 687)
(685, 686)
(694, 695)
(705, 706)
Total number of pairs:
43


Similar Pairs: (11, 19)
(22, 47)
(48, 87)
(51, 83)
(104, 126)
(117, 121)
(123, 129)
(124, 132)
(125, 133)
(141, 214)
(142, 143)
(152, 153)
(160, 161)
(187, 188)
(226, 486)
(227, 434)
(242, 251)
(311, 312)
(356, 362)
(382, 385)
(387, 388)
(413, 429)
(495, 622)
(506, 521)
(508, 520)
(518, 632)
(534, 535)
(538, 550)
(541, 624)
(591, 593)
(601, 602)
(633, 634)
(640, 727)
(641, 649)
(643, 725)
(646, 652)
(648, 654)
(650, 731)
(660, 674)
(683, 687)
(685, 686)
(694, 695)
(705, 706)

Total number of pairs:
43

## 4.2 Comparison

In [66]:
# Comparison

# True Positives:
TP = true_positive(similar_pairs, similar_pairs_approximate)
print("True Positives: {}".format(TP))

# False Positives:
FP = false_positive(similar_pairs, similar_pairs_approximate)
print("False Positives: {}".format(FP))

# False Negatives:
FN = false_negative(similar_pairs, similar_pairs_approximate)
print("False Negatives: {}".format(FN))

# True Negatives:
TN = len(similarity_mtx) - TP - FN - FP
print("True Negatives: {}".format(TN))

# F1 score of minhashing approximation
f1_score = (2*TP)/(2*TP+FN+FP)
print("F1 score of approximation: {}".format(f1_score))


True Positives: 41
False Positives: 2
False Negatives: 0
True Negatives: 271173
F1 score of approximation: 0.9761904761904762


True Positives: 41

False Positives: 2

False Negatives: 0

True Negatives: 271173

F1 score of approximation: 0.9761904761904762

# Step 5: Observation

- Using minhashing to represent the original sparse matrix with signiture vectors greatly improved the efficiency of similarity measurement, reducing the time consumption from nearly 5 minutes to less than 20 seconds (on my device).

- Using 50 hash functions to represent the signiture of documents can already achieve an f1 score of around 0.95 (fluctuates with random parameters), while efficiently compressing the original 4613x737 matrix to 50x737.

- Using 100 hash functions can achieve around 0.96~1.00 f1 score, while compressing the matrix to 100x737.